In [37]:
import pandas as pd
import datetime
import os
from jinja2 import Environment, PackageLoader, select_autoescape, FileSystemLoader
import subprocess

In [33]:
env = Environment(
    loader=FileSystemLoader('.'),
    trim_blocks = True,
    autoescape=select_autoescape(['html', 'ipynb'])
)

In [40]:
process = subprocess.Popen(['git', 'rev-parse', 'HEAD'], shell=False, stdout=subprocess.PIPE, cwd='data1')
git_head_hash = process.communicate()[0].strip()

In [34]:
with open('plotting_snia_toy06_test_template.ipynb') as fh:
    template_str = fh.read()
with open('plotting_model_template.ipynb.j2', 'w') as fh:
    fh.write(template_str.replace('toy06', '{{ model_name }}'))

In [35]:
plotting_template = env.get_template('plotting_model_template.ipynb.j2')

In [36]:
for model in ['toy01', 'toy06', 'ddc10', 'ddc25']:
    nb_fname = 'plotting_{0}.ipynb'.format(model)
    with open(nb_fname, 'w') as fh:
        fh.write(plotting_template.render({'model_name': model, 
                                           'time_generated':str(datetime.datetime.now()),
                                           'data1_git_hash':git_head_hash}))
    os.system('jupyter-nbconvert {0} --allow-errors --clear-output --inplace --execute --to notebook'.format(nb_fname))
    os.system('jupyter-nbconvert {0} --no-input'.format(nb_fname))

In [11]:
!mv *.html ../sn-rad-trans.github.io/